In [ ]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
#Connect to local postgres database
conn = psycopg2.connect(database="delinquency_model", user="michael" , password="Azn4life!" , host="127.0.0.1") #ommitted login information for privacy
cur = conn.cursor()

In [ ]:
#Query to create and populate tables(loans, borrower)
#Tables are in localhost database called delinquency_model
create_table_query = """
DROP TABLE IF EXISTS borrower;
CREATE TABLE borrower (
    Account_Number integer,
    First_Name varchar(100),
    Last_Name varchar(100),
    Zip_Code INTEGER,
    Country_Code varchar(50),
    Income_Annual money,
    Total_Mortgage_Balances integer,
    Total_HE_Balances integer,
    Num_Trades_Past_Due integer,
    Num_He_Trades integer   
);

DROP TABLE IF EXISTS loans;
CREATE TABLE loans (
    Account_Number integer,
    Month timestamp,
    Product varchar(100),
    FICO_Score_Original integer,
    FICO_Score_Current integer,
    Origination_Date timestamp,
    Maturity_Date timestamp,
    Commitments decimal,
    Outstandings decimal,
    LTV_Original decimal,
    LTV_Current decimal,
    Lien_Position varchar(100),
    Day_Past_Due integer,
    Interest_Variability_Code varchar(50),
    Collateral_Value money,
    Payment_Amount money,
    Origination_DTI decimal,
    APR decimal
    
);

COPY borrower FROM '/Users/michaeltan/dev/venv/HELOAN Delinquency Model/Data/borrower.csv' DELIMITER ',' CSV HEADER;
COPY loans FROM '/Users/michaeltan/dev/venv/HELOAN Delinquency Model/Data/loans.csv' DELIMITER ',' CSV HEADER;

"""

In [ ]:
#Execute create_table_query
cur.execute(create_table_query)

In [ ]:
#Query to clean data
query = '''
--Add new column with days_past_due of next month, used as delinquency training outcome
--join borrower info onto each account 
WITH loans2 AS (

SELECT 
    a.*,
    b.day_past_due as day_past_due_next_month,
    c.total_mortgage_balances,
    c.total_he_balances,
    c.num_trades_past_due,
    c.num_he_trades
    
FROM loans as a
LEFT JOIN loans as b
    ON a.account_number = b.account_number
        AND date_trunc('month', a.month) = date_trunc('month', b.month - interval '1' month)
LEFT JOIN borrower as c
    ON a.account_number = c.account_number
    
),

--clean rows of empty or missing data
clean_rows AS (

SELECT 
    *
FROM loans2
WHERE COALESCE(commitments, apr, fico_score_current, ltv_current) != 0 --no line of credit so not relevant to predicting current customers. will keep APR because high apr represents high risk
    AND COALESCE(loans2.*) is not null
    
),



--Only select columns used as predictors
--Turn day_past_due_next_month into 1(yes) or 0(no) and remove last row with no next month info
--Add column called utilization which = outstandings/commitments
loans3 AS (

SELECT 
    account_number,
    month,
    fico_score_current,
    commitments,
    outstandings,
    (outstandings/commitments) as utilization,
    ltv_current,
    origination_dti,
    apr,
    total_mortgage_balances,
    total_he_balances,
    num_trades_past_due,
    num_he_trades,
    CASE WHEN day_past_due_next_month = 0 THEN 0
        ELSE 1
    END AS delinquent_next_month
FROM clean_rows
WHERE day_past_due_next_month is not null 

)

SELECT * FROM loans3
ORDER BY account_number desc

;
'''

In [ ]:
#Store results of query in a dataframe
train_data = pd.read_sql(query, con = conn)
train_data

In [ ]:
print(train_data.describe())

In [ ]:
#Normalize data because orders of magnitude between variables are very different
X = train_data.iloc[:,2:13] #independent columns
y = train_data.iloc[:,13] #response column
normalized_X = preprocessing.normalize(X)

In [ ]:
#Univariate feature selection

#apply SelectKBest to rank the features
features = SelectKBest(score_func = chi2, k = 'all')
fit = features.fit(normalized_X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

#concat two dataframes for visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1) 
featureScores.columns = ['feature','Score']  #naming the dataframe columns
print(featureScores.nlargest(11,'Score')) # scores is ratio explained over unexplained

In [ ]:
#Horizontal barplot of feature scores

# Get the indices sorted by most important to least important
indices = np.argsort(fit.scores_)[::-1]

# top feature names
features = []
for i in range(11):
    features.append(X.columns[indices[i]])

#Plot using seaborn on log scale
fig, ax = plt.subplots()
ax.set_xscale('log')
ax.invert_yaxis()
ax.set(xlabel='Scores', ylabel='Features', title = 'Feature Scores Ranked')
sns.barplot(fit.scores_[indices[range(11)]], features, palette='Blues_d', orient = 'h', ax = ax)

In [ ]:
#Logistic Regression with GridSearchCV tuning (fitting all possible values)
X = train_data.iloc[:,[2, 3, 4, 6, 7, 9, 11]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
clf = GridSearchCV(LogisticRegression(penalty='l2', class_weight = 'balanced'), param_grid)



clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Best score: %.3f" % clf.score(X_test, y_test))
print (classification_report(y_test, y_pred))
    
count = pd.DataFrame(y_test)
count['delinquent_next_month'].value_counts()

In [ ]:
#Confusion matrix, predicted vs actual, heatmap
cnf_matrix = confusion_matrix(y_test,y_pred)

class_names=['current','delinquent'] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual', rotation = 0)
plt.xlabel('Predicted')

In [ ]:
#implementation of model
test_data = pd.read_csv() #insert csv file here. Column format: (account_number', 'month', 'fico_score_current', 'commitments', 'outstandings', 'utilization', 'ltv_current', 'origination_dti', 'apr', 'total_mortgage_balances', 'total_he_balances', 'num_trades_past_due', 'num_he_trades')
prediction = clf.predict(test_data)
submission = pd.DataFrame({
    'account_number': test_data['account_number'],
    'delinquent': prediction
})
submission.to_csv('submission.csv', index=False)